In [ ]:
%%html
<script>
(function() {
  // Create the toggle button
  const rtlButton = document.createElement("button");
  rtlButton.textContent = "Toggle LTR";
  rtlButton.id = "top-rtl-toggle";
  rtlButton.style.marginLeft = "8px";
  rtlButton.style.padding = "4px 10px";
  rtlButton.style.fontSize = "14px";
  rtlButton.style.cursor = "pointer";

  // State
  var rtlActive = false;

  // Styling function
  var applyStyleToEditor = (editor) => {
    if (!editor) return;
    var direction = getComputedStyle(editor).getPropertyValue('direction')=='rtl' ? 'ltr' : 'rtl';
    var text_align = getComputedStyle(editor).getPropertyValue('text-align')=='right' ? 'left' : 'right';
    editor.style.setProperty('direction', direction, 'important');
    editor.style.setProperty('text-align', text_align, 'important');
  };

  // Toggle logic
  rtlButton.onclick = () => {
    rtlActive = !rtlActive;
    rtlButton.textContent = rtlActive ? "Toggle LTR" : "Toggle RTL";
    document.querySelectorAll('.jp-MarkdownCell .jp-InputArea-editor').forEach(applyStyleToEditor);
    document.querySelectorAll('.jp-RenderedHTMLCommon code, .jp-RenderedHTMLCommon code span').forEach(applyStyleToEditor);
    document.querySelectorAll('jp-RenderedHTMLCommon, .jp-RenderedHTMLCommon *').forEach(applyStyleToEditor);
  };

  // Watch for focus into editing Markdown cells
  // document.addEventListener('focusin', (event) => {
  //   const editor = event.target.closest('.jp-MarkdownCell .jp-InputArea-editor');
  //    if (editor) applyStyleToEditor(editor);
  // });

  // Insert into top toolbar if not already present
  var insertIntoToolbar = () => {
    const toolbar = document.querySelector('.jp-NotebookPanel-toolbar');
    if (toolbar && !document.getElementById("top-rtl-toggle")) {
      toolbar.appendChild(rtlButton);
    } else {
      // Try again in a moment if toolbar isn't ready yet
      setTimeout(insertIntoToolbar, 300);
    }
  };

  insertIntoToolbar();
})();
</script>

In [ ]:
%%html
<!-- <style>
  table {display: inline-block}
</style> -->

# שבוע 9 - Matplotlib & SciPy


# %% [markdown]
# חלק א' — Matplotlib (≈ 1/3)

## 1. אנטומיית תרשימים
**מושגים מרכזיים:**  
- **Figure** — מכלול התרשים (הדף).  
- **Axes** — מערכת צירים אחת (או יותר) בתוך ה־Figure.  
- **Axis** — ציר $x$ או $y$ בתוך ה־Axes.

יש שתי דרכים עיקריות לעבוד:  
1. **סגנון Pyplot** (מהיר): `plt.plot(...)`, `plt.show()`  
2. **מודל אובייקטי (מומלץ לפרויקטים)**: יצירת `fig, ax = plt.subplots()` ואז שימוש ב־`ax.plot(...)` וכן הלאה.

נוסיף כותרות, תוויות, ונטמיע נוסחאות עם $ \LaTeX $ בכותרת/צירים.
ראשית, נייבא את הספרייה ונאתחל פרמטרים

In [ ]:
# Core imports used across the notebook
import numpy as np
import matplotlib.pyplot as plt

# For SciPy sections:
from scipy import integrate, optimize, interpolate, fft, signal, linalg, stats, constants

# Global plotting defaults (feel free to tweak)
plt.rcParams.update({
    "figure.figsize": (7, 4.5),
    "figure.dpi": 120,
    "axes.grid": True,
    "font.size": 11
})


כעת ניצור גרף פשוט 

In [ ]:
# %%
# Quickstart: object-oriented API
x = np.linspace(0, 2*np.pi, 200)
y = np.sin(x)

fig, ax = plt.subplots()
ax.plot(x, y, label="sin(x)")
ax.set_title(r"גל סינוס: $y=\sin(x)$")
ax.set_xlabel(r"$x$ (rad)")
ax.set_ylabel(r"$y$")
ax.legend()
plt.tight_layout()
plt.show()


### תצוגה, גודל, ופריסות
- **גודל ו-DPI**: אפשר לקבוע יחס גובה/רוחב וחדות.  
- **`tight_layout()` / `constrained_layout=True`**: דחיסה חכמה של רכיבי התרשים.  
- **`savefig`**: שמירת תרשימים ל־PNG/SVG/PDF באיכות הגשה.

In [ ]:
# %%
# Figure size, dpi, layout, and saving
fig, ax = plt.subplots(figsize=(6, 3.8), constrained_layout=True)
ax.plot(x, np.cos(x), label="cos(x)")
ax.set_title(r"גל קוסינוס: $y=\cos(x)$")
ax.set_xlabel(r"$x$ (rad)")
ax.set_ylabel(r"$y$")
ax.legend(loc="upper right")

# Uncomment to save a publication-quality figure:
# fig.savefig("cosine_plot.pdf")  # vector format (great for print)
plt.show()


# %% [markdown]
## 2. סוגי גרפים שכיחים בפיזיקה
נראה מספר סוגים שימושיים:
- **קווי**: `plot` להשוואת מערכי נתונים;  
- **פיזור**: `scatter` לקשר בין שתי מדידות;  
- **היסטוגרמה**: `hist` לבדיקת התפלגות;  
- **מפה/תמונה**: `imshow`/`matshow` להצגת שדה (למשל טמפרטורה על רשת);  
- **שגיאות**: `errorbar` למידע ניסויי עם אי־ודאות.


In [ ]:
# %%
# Lines vs scatter vs hist vs imshow vs errorbar — compact demo
rng = np.random.default_rng(42)
x = np.linspace(0, 10, 50)
y_true = 2.0 * x + 1.0
y_noisy = y_true + rng.normal(0, 2.0, size=x.size)

fig, axs = plt.subplots(2, 3, figsize=(10, 6), constrained_layout=True)

# Line
axs[0,0].plot(x, y_true, label="true")
axs[0,0].plot(x, y_noisy, ".", label="noisy")
axs[0,0].set_title("Line")
axs[0,0].legend()

# Scatter
axs[0,1].scatter(x, y_noisy)
axs[0,1].set_title("Scatter")

# Histogram
data = rng.normal(loc=0.0, scale=1.0, size=1000)
axs[0,2].hist(data, bins=30)
axs[0,2].set_title("Histogram")

# Imshow (heatmap-like)
grid_x = np.linspace(-2, 2, 50)
grid_y = np.linspace(-2, 2, 50)
X, Y = np.meshgrid(grid_x, grid_y)
Z = np.exp(-(X**2 + Y**2))
im = axs[1,0].imshow(Z, extent=[grid_x.min(), grid_x.max(), grid_y.min(), grid_y.max()],
                     origin="lower", aspect="auto")
axs[1,0].set_title("Imshow (Gaussian field)")
fig.colorbar(im, ax=axs[1,0], shrink=0.8)

# Errorbar
yerr = rng.uniform(0.5, 1.5, size=x.size)
axs[1,1].errorbar(x, y_noisy, yerr=yerr, fmt="o")
axs[1,1].set_title("Errorbar")

# Empty slot for future (e.g., bar or boxplot)
axs[1,2].text(0.5, 0.5, "Use this slot for extra plot", ha="center", va="center")
axs[1,2].set_axis_off()

plt.show()


# %% [markdown]
## 3. התאמות מתקדמות: סולמות, טיקים, מקרא, סימונים, צירים תאומים, colormaps, ושמירה
- **סולמות לוגריתמיים**: `ax.set_xscale('log')`, `ax.set_yscale('log')` — שימושי לתופעות המוצגות על פני סדרי גודל.  
- **טיקים ותוויות**: שליטה בתצוגת מספרים (מדעי/אקספוננציאלי), והוספת יחידות.  
- **מקרא ו־annotations**: הוספת הסברים וחצים לנקודות חשובות.  
- **ציר תאום**: `ax.twinx()` להצגת שתי יחידות שונות על אותו גרף.  
- **Colormaps**: בחירה מודעת למידע מדעי (למשל `viridis`, `inferno`).

In [ ]:
# %%
# Log scales, annotations, twin axis
x = np.logspace(-2, 2, 200)
y = 1 / (1 + x**2)

fig, ax1 = plt.subplots(constrained_layout=True)
ax1.set_xscale("log")
ax1.plot(x, y, label=r"$y=\frac{1}{1+x^2}$")
ax1.set_xlabel(r"$x$ (log scale)")
ax1.set_ylabel(r"$y$")
ax1.legend(loc="upper right")

# Annotation
idx = np.argmin(np.abs(x - 1.0))
ax1.annotate("knee ~ x=1", xy=(x[idx], y[idx]), xytext=(0.3, 0.7),
             textcoords="axes fraction",
             arrowprops=dict(arrowstyle="->"))

# Twin axis (for demonstration)
ax2 = ax1.twinx()
ax2.plot(x, y*np.pi, linestyle="--")
ax2.set_ylabel(r"$\pi y$ (twin)")

plt.show()


# %% [markdown]
### שאלות רב־ברירה — חלק א' (Matplotlib)
1. אם ברצוננו להציג נתונים על פני שלושה סדרי גודל בציר $x$, הבחירה הטובה היא:  
   A. `ax.set_xscale('linear')`  
   B. `ax.set_xscale('log')`  
   C. `ax.set_yscale('log')`  
   D. להשאיר ברירת מחדל  
2. מה יוצר אובייקט צירים חדש לשיטה המומלצת?  
   A. `fig = plt.figure()`  
   B. `ax = plt.axes()`  
   C. `fig, ax = plt.subplots()`  
   D. `plt.plot(...)`  
3. להצגת שדה דו־ממדי (כמו טמפרטורה על רשת), מה מתאים?  
   A. `plot`  
   B. `scatter`  
   C. `hist`  
   D. `imshow`

### איור (סקיצה)
- מקמו כאן תמונה/איור סכמטי שמסביר את ההבדל בין Figure, Axes, Axis (אפשר לצייר ידנית ולהדביק).


# %% [markdown]
## חלק ב' — SciPy (≈ 2/3)

## 1. מה זה SciPy + תזכורת קצרה ל-NumPy + קבועים פיזיקליים
**SciPy** מוסיפה לאקוסיסטם של NumPy כלים נומריים מתקדמים: אינטגרציה, פתרון ODEs, אופטימיזציה, אינטרפולציה, FFT, עיבוד אותות, אלגברה ליניארית, סטטיסטיקה ועוד.  
**קבועים פיזיקליים** זמינים ב־`scipy.constants`: למשל $c, h, k_B, G$ וכד'.

נזכיר וקטוריזציה ו־broadcasting: פעולות מתמטיות על מערכי NumPy ללא לולאות פייתון איטיות (ברוב המקרים).


In [ ]:
# %%
# Constants and quick NumPy recap
c = constants.c        # speed of light (m/s)
h = constants.h        # Planck constant (J s)
kB = constants.Boltzmann  # Boltzmann (J/K)

# Vectorized operations example
x = np.linspace(0, 1, 5)
y = x**2 + 2*x + 1  # y = (x+1)^2
print("x:", x)
print("y:", y)
print("Some constants:", {"c": c, "h": h, "kB": kB})


# %% [markdown]
### שאלות רב־ברירה
1. היכן נמצא $k_B$?  
   A. `numpy.constants.kB`  
   B. `scipy.constants.Boltzmann`  
   C. `scipy.constants.k`  
   D. `math.constants.kB`  
2. מהי וקטוריזציה?  
   A. חישוב רק עבור וקטורים מאורך 3  
   B. שימוש ב־for כפול בלולאה  
   C. ביצוע פעולות על מערכים שלמים ללא לולאות פייתון  
   D. שימוש ב־`vector` ממודול `collections`

### איור (סקיצה)
- תרשים סכמטי של זרימת נתונים: מדידה → NumPy (עיבוד) → SciPy (ניתוח) → Matplotlib (הצגה).


# %% [markdown]
## 2. אינטגרציה וגזירה נומרית + ODEs
**אינטגרציה חד־ממדית:** `integrate.quad` עבור $\int_a^b f(x)\,dx$.  
**טרפזים/מצטבר:** `numpy.trapz`, `scipy.integrate.cumtrapz` כדגמים פשוטים.  
**משוואות דיפרנציאליות רגילות (ODE):**  
נפתור בעיה כללית $ \dot{\mathbf{y}} = f(t,\mathbf{y}) $ עם `solve_ivp`, ונראה דוגמת מתנד הרמוני $ \ddot{x} + \omega^2 x = 0 $.



In [ ]:
# %%
# Numeric integration: quad vs trapz
f = lambda x: np.exp(-x**2)  # Gaussian

I_quad, err = integrate.quad(f, 0, np.inf)  # integral from 0 to infinity
xs = np.linspace(0, 4, 1000)
I_trapz = np.trapz(f(xs), xs)

print("quad integral ~", I_quad, "estimated error ~", err)
print("trapz integral ~", I_trapz)


In [ ]:
# %%
# ODE: simple harmonic oscillator x'' + w^2 x = 0
w = 2*np.pi  # rad/s

def sho(t, y):
    # y = [x, v]
    x, v = y
    dxdt = v
    dvdt = -w**2 * x
    return [dxdt, dvdt]

t_span = (0, 2.0)        # seconds
y0 = [1.0, 0.0]          # initial displacement 1, zero velocity
sol = integrate.solve_ivp(sho, t_span, y0, dense_output=True, max_step=0.01)

t = np.linspace(*t_span, 500)
x = sol.sol(t)[0]

fig, ax = plt.subplots()
ax.plot(t, x)
ax.set_title(r"מתנד הרמוני: $x''+\omega^2 x=0$")
ax.set_xlabel("t (s)")
ax.set_ylabel("x (m)")
plt.show()


# %% [markdown]
### שאלות רב־ברירה
1. איזו פונקציה תחשב $\int_0^\infty e^{-x^2}\,dx$ בדיוק נומרי טוב?  
   A. `np.sum`  
   B. `integrate.quad`  
   C. `np.mean`  
   D. `fft.rfft`  
2. עבור $x''+\omega^2 x=0$, מה נשתמש בו ל־ODE?  
   A. `optimize.minimize`  
   B. `interpolate.interp1d`  
   C. `integrate.solve_ivp`  
   D. `stats.linregress`

### איור (סקיצה)
- שרטטו סכמת וקטור מצב $\mathbf{y}=[x,v]$ והקשר בין נגזרות לזמן.


# %% [markdown]
## 3. אופטימיזציה, שורשים, והתאמת עקומות
**שורשים:** `optimize.root_scalar` לאיתור $x$ כך ש־$f(x)=0$.  
**מינימום:** `minimize_scalar` (חד־ממדי), `minimize` (רב־ממדי).  
**התאמת עקומות ניסוי:** `optimize.curve_fit` — התאמה לא־ליניארית לנתונים $(x,y)$ והפקת טבלת covariance להערכת אי־ודאות.

נציג התאמה לקו $y=ax+b$ ונדגים הוצאת שגיאות על $a,b$.


In [ ]:
# %%
# Root finding example: solve x*cos(x) = 0 near x ~ pi/2
g = lambda x: x*np.cos(x)
root = optimize.root_scalar(g, bracket=(0.1, 2.0))
print("root near (0.1,2.0):", root.root)

# Curve fitting example: y = a*x + b
rng = np.random.default_rng(0)
x = np.linspace(0, 10, 50)
a_true, b_true = 2.0, 1.0
y = a_true*x + b_true + rng.normal(0, 1.2, size=x.size)

def linear_model(x, a, b):
    return a*x + b

popt, pcov = optimize.curve_fit(linear_model, x, y)
a_hat, b_hat = popt
perr = np.sqrt(np.diag(pcov))

print(f"Estimated a={a_hat:.3f} ± {perr[0]:.3f}, b={b_hat:.3f} ± {perr[1]:.3f}")

fig, ax = plt.subplots()
ax.errorbar(x, y, yerr=1.2, fmt="o", label="data")
ax.plot(x, linear_model(x, *popt), label="fit")
ax.set_title("Linear fit with curve_fit")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.legend()
plt.show()


# %% [markdown]
### שאלות רב־ברירה
1. מה מחזירה `curve_fit` בנוסף לפרמטרים?  
   A. מטריצת ג'ייקוביאן  
   B. טבלת covariance של הפרמטרים  
   C. גרף שיוריים  
   D. שום דבר נוסף  
2. למציאת $x$ כך ש־$f(x)=0$ במרווח ידוע, נעדיף:  
   A. `root_scalar` עם `bracket=(a,b)`  
   B. `minimize`  
   C. `interp1d`  
   D. `rfft`

### איור (סקיצה)
- סכימה של שיוריים $r_i=y_i - \hat{y}_i$ והמשמעות שלהם בבדיקת איכות התאמה.


# %% [markdown]
## 4. אינטרפולציה והחלקה
**אינטרפולציה חד־ממדית:** `interpolate.interp1d(x, y, kind='linear'|'cubic'|...)` — קירוב ערכים בין נקודות מדידה.  
**Spline**: `UnivariateSpline` מאפשר החלקה (smoothing) באמצעות פרמטר $s$.



In [ ]:
# %%
# Interpolation demo
x = np.linspace(0, 10, 11)
y = np.sin(x) + 0.1*np.cos(3*x)
f_lin = interpolate.interp1d(x, y, kind="linear")
f_cub = interpolate.interp1d(x, y, kind="cubic")

xf = np.linspace(0, 10, 400)

fig, ax = plt.subplots(constrained_layout=True)
ax.plot(x, y, "o", label="samples")
ax.plot(xf, f_lin(xf), label="linear")
ax.plot(xf, f_cub(xf), label="cubic")
ax.set_title("Interpolation: linear vs cubic")
ax.legend()
plt.show()


In [ ]:
# %%
# Spline smoothing demo
x = np.linspace(0, 10, 80)
rng = np.random.default_rng(1)
y = np.sin(x) + 0.4*rng.normal(size=x.size)

spline = interpolate.UnivariateSpline(x, y, s=5.0)  # smoothing parameter
xf = np.linspace(0, 10, 400)

fig, ax = plt.subplots()
ax.plot(x, y, ".", label="noisy samples")
ax.plot(xf, spline(xf), "-", label="smoothed spline")
ax.set_title("Smoothing with UnivariateSpline")
ax.legend()
plt.show()
    

# %% [markdown]
### שאלות רב־ברירה
1. איזה פרמטר ב־`UnivariateSpline` שולט על החלקה?  
   A. `k`  
   B. `s`  
   C. `kind`  
   D. `bounds_error`  
2. באינטרפולציה קובית, `kind=` צריך להיות:  
   A. `"cubic"`  
   B. `"quadratic"`  
   C. `"spline"`  
   D. `"poly3"`

### איור (סקיצה)
- סכימה של דגימות בדידות, אינטרפולציה ליניארית לעומת קובית, והחלקת spline.


# %% [markdown]
## 5. עיבוד אותות ו-FFT
**FFT חד־צדדי:** `fft.rfft` לאותים ממשיים; תדירויות ב־`fft.rfftfreq(N, d=1/fs)`.  
**נייקוויסט:** $f_N = \tfrac{f_s}{2}$.  
**החלקה בסיסית:** `signal.savgol_filter`.  
**איתור פסגות:** `signal.find_peaks`.

ננתח אות סינוס מרובע תדרים ועם רעש, נזהה פסגות תדר.


# %% [markdown]
## 5. עיבוד אותות ו-FFT
**FFT חד־צדדי:** `fft.rfft` לאותים ממשיים; תדירויות ב־`fft.rfftfreq(N, d=1/fs)`.  
**נייקוויסט:** $f_N = \tfrac{f_s}{2}$.  
**החלקה בסיסית:** `signal.savgol_filter`.  
**איתור פסגות:** `signal.find_peaks`.

ננתח אות סינוס מרובע תדרים ועם רעש, נזהה פסגות תדר.


In [ ]:
# %%
# Signal + FFT + peak finding
fs = 500.0  # Hz
T = 2.0     # seconds
t = np.arange(0, T, 1/fs)
rng = np.random.default_rng(2)
sig = 0.7*np.sin(2*np.pi*50*t) + 0.3*np.sin(2*np.pi*120*t) + 0.2*rng.normal(size=t.size)

# FFT
yf = fft.rfft(sig)
ff = fft.rfftfreq(t.size, d=1/fs)
amp = np.abs(yf)/t.size

# Smooth time signal (optional)
sig_smooth = signal.savgol_filter(sig, window_length=51, polyorder=3)

# Peak detection on spectrum
peaks, _ = signal.find_peaks(amp, height=np.max(amp)*0.3)

fig1, ax1 = plt.subplots()
ax1.plot(t, sig, label="signal")
ax1.plot(t, sig_smooth, label="smoothed")
ax1.set_title("Time signal")
ax1.set_xlabel("t (s)")
ax1.legend()

fig2, ax2 = plt.subplots()
ax2.plot(ff, amp, label="spectrum")
ax2.plot(ff[peaks], amp[peaks], "x", label="peaks")
ax2.set_xlim(0, 200)
ax2.set_xlabel("f (Hz)")
ax2.set_ylabel("Amplitude")
ax2.set_title("Amplitude spectrum")
ax2.legend()
plt.show()


# %% [markdown]
### שאלות רב־ברירה
1. תדירות נייקוויסט היא:  
   A. $f_s$  
   B. $2f_s$  
   C. $f_s/2$  
   D. $1/f_s$  
2. איזו פונקציה מתאימה לאיתור פסגות בספקטרום?  
   A. `fft.rfft`  
   B. `signal.find_peaks`  
   C. `integrate.quad`  
   D. `stats.linregress`

### איור (סקיצה)
- גרף סכמטי של דגימה בזמן וייצוגה בתחום התדר (FFT).


# %% [markdown]
## 6. אלגברה ליניארית מדעית
נפתור $A\mathbf{x}=\mathbf{b}$, ונחשב ערכים עצמיים/וקטורים עצמיים:  
- פתרון מערכת: `linalg.solve(A, b)`  
- ערכים עצמיים: `linalg.eig(A)` (כללי), `linalg.eigh(A)` (סימטרי/הרמי)



In [ ]:
# %%
# Linear system + eigenvalues/eigenvectors
A = np.array([[3.0, 1.0], [1.0, 2.0]])
b = np.array([9.0, 8.0])
x = linalg.solve(A, b)

w, V = linalg.eig(A)     # eigenvalues & eigenvectors

print("Solution x:", x)
print("Eigenvalues:", w)
# V columns are eigenvectors


# %% [markdown]
### שאלות רב־ברירה
1. עבור מטריצה סימטרית ממשית, נעדיף:  
   A. `eig`  
   B. `eigh`  
   C. `svd`  
   D. `solve`  
2. פתרון $A\mathbf{x}=\mathbf{b}$ נעשה עם:  
   A. `linalg.solve`  
   B. `optimize.minimize`  
   C. `fft.rfft`  
   D. `stats.ttest_ind`

### איור (סקיצה)
- המחשת כיווני וקטורים עצמיים והמתיחות לאורך כיוונים אלו.


# %% [markdown]
## 7. סטטיסטיקה בסיסית למדידות
נדגים שימוש בתפלגות נורמלית, אומדן פרמטרים, והתאמת התפלגות לנתונים:  
- `stats.norm` עבור PDF/CDF ודגימה אקראית.  
- אומדן $\mu,\sigma$ מנתונים, והשוואה לרגרסיה/התאמה.



In [ ]:
# %%
# Basic stats with normal distribution
rng = np.random.default_rng(123)
samples = rng.normal(loc=2.0, scale=0.5, size=1000)

mu_hat = np.mean(samples)
sigma_hat = np.std(samples, ddof=1)

# PDF vs histogram
xs = np.linspace(mu_hat-3*sigma_hat, mu_hat+3*sigma_hat, 300)
pdf = stats.norm.pdf(xs, loc=mu_hat, scale=sigma_hat)

fig, ax = plt.subplots()
ax.hist(samples, bins=30, density=True, alpha=0.6, label="data")
ax.plot(xs, pdf, label=rf"Normal($\mu$={mu_hat:.2f}, $\sigma$={sigma_hat:.2f})")
ax.set_title("Histogram vs Normal PDF")
ax.legend()
plt.show()


# %% [markdown]
### שאלות רב־ברירה
1. מה ההבדל בין `np.std(..., ddof=0)` ו-`ddof=1`?  
   A. אין הבדל  
   B. `ddof=1` נותן אומדן חסר־הטיה לסטיית תקן במדגם  
   C. `ddof=0` גדול יותר תמיד  
   D. שניהם מחזירים שונות ולא סטיית תקן  
2. `stats.norm.pdf(x, loc, scale)` מחזירה:  
   A. פונקציית התפלגות מצטברת  
   B. צפיפות הסתברות נקודתית  
   C. דגימות אקראיות  
   D. סטיית תקן

### איור (סקיצה)
- המחשה של $68$–$95$–$99.7\%$ סביב $\mu \pm \sigma, 2\sigma, 3\sigma$.


# %% [markdown]
# דוגמאות מסכמות (Capstone)

## דוגמה 1 — מטוטלת: הערכת $g$ מהתלות $T(L)$
**רקע:** עבור זוויות קטנות, תקופת מטוטלת מאורך $L$ היא בקירוב $ T(L) \approx 2\pi\sqrt{\tfrac{L}{g}} $.  
**מטרה:** בהתבסס על מדידות $(L_i, T_i)$ עם אי־ודאות, נאמוד את $g$ בעזרת התאמה לא־ליניארית, נציג פסי שגיאה, ונבחן שיוריים.

שלבים:  
1. יצירת/טעינת נתוני ניסוי עם רעש.  
2. התאמה עם `curve_fit` למודל $T(L)=a\sqrt{L}$ כאשר $a\approx 2\pi/\sqrt{g}$ או ישירות פרמטר $g$.  
3. פסי שגיאה, שיוריים, וטווחי ביטחון.



In [ ]:
# %%
# Pendulum: estimate g from T(L)
rng = np.random.default_rng(0)
g_true = 9.81

# Synthetic measurements
L = np.linspace(0.2, 1.2, 10)          # meters
T_true = 2*np.pi*np.sqrt(L/g_true)
sigma_T = 0.02 + 0.01*np.sqrt(L)       # simple nonuniform uncertainty
T_meas = T_true + rng.normal(0, sigma_T)

def T_model(L, g):
    return 2*np.pi*np.sqrt(L/g)

popt, pcov = optimize.curve_fit(T_model, L, T_meas, sigma=sigma_T, absolute_sigma=True, p0=[9.5])
g_hat = popt[0]
g_err = np.sqrt(np.diag(pcov))[0]

# Residuals
res = T_meas - T_model(L, g_hat)

# Plot
fig, axs = plt.subplots(2, 1, figsize=(6.5, 6), constrained_layout=True)
axs[0].errorbar(L, T_meas, yerr=sigma_T, fmt="o", label="measurements")
L_f = np.linspace(L.min(), L.max(), 300)
axs[0].plot(L_f, T_model(L_f, g_hat), label=rf"fit: $g={g_hat:.3f}\pm{g_err:.3f}\ \mathrm{{m/s^2}}$")
axs[0].set_xlabel("L (m)")
axs[0].set_ylabel("T (s)")
axs[0].set_title("Pendulum period vs length")
axs[0].legend()

axs[1].axhline(0, color="k", linewidth=1)
axs[1].plot(L, res, "o-")
axs[1].set_xlabel("L (m)")
axs[1].set_ylabel("Residual (s)")
axs[1].set_title("Residuals")
plt.show()


# %% [markdown]
### שאלות רב־ברירה (דוגמה 1)
1. אם $T=a\sqrt{L}$, אז $a$ קשור ל־$g$ כך ש:  
   A. $a = \dfrac{1}{2\pi}\sqrt{g}$  
   B. $a = 2\pi\sqrt{g}$  
   C. $a = \dfrac{2\pi}{\sqrt{g}}$  
   D. $a = g$  
2. מה תפקיד וקטור `sigma` ב־`curve_fit`?  
   A. לקבוע צבעים בגרף  
   B. להגדיר משקלות להתאמה בהתאם לאי־ודאות  
   C. להאיץ את החישוב  
   D. להחליף את הפונקציה למודל

### איור (סקיצה)
- סכמת ניסוי מטוטלת: נקודת תלייה, חוט באורך $L$, סטייה קטנה, מדידת תקופה.


# %% [markdown]
## דוגמה 2 — קליע עם גרר ריבועי: זווית שיגור מיטבית
**רקע:** כוח גרר ריבועי: $ \mathbf{F}_d = -\tfrac{1}{2}\rho C_d A v \mathbf{v} $.  
שקול ל־ODE דו־ממדי עבור $(x,y)$ עם מהירויות $(v_x,v_y)$:  
\[
\begin{aligned}
\dot{x}&=v_x,\quad \dot{y}=v_y,\\
\dot{v}_x&=-\frac{\rho C_d A}{2m}\,v\,v_x,\quad
\dot{v}_y=-g-\frac{\rho C_d A}{2m}\,v\,v_y,
\end{aligned}
\]
כאשר $v=\sqrt{v_x^2+v_y^2}$.

**מטרה:** לפתור את ה־ODE עד פגיעה בקרקע (אירוע אפס ב־$y$), ולמצוא ע"י `minimize_scalar` את הזווית $\theta$ שממקסמת את הטווח האופקי.


In [ ]:
# %%
# Projectile with quadratic drag: optimal launch angle
g = 9.81
rho = 1.225     # air density (kg/m^3)
CdA = 0.05      # drag coefficient * area (m^2)
m = 0.145       # mass (kg) ~ baseball
v0 = 40.0       # m/s

def flight(theta_deg):
    # ODE system with quadratic drag
    theta = np.deg2rad(theta_deg)
    vx0 = v0*np.cos(theta)
    vy0 = v0*np.sin(theta)

    def ode(t, y):
        # y=[x, y, vx, vy]
        x, y_, vx, vy = y
        v = np.hypot(vx, vy)
        ax = -(rho*CdA/(2*m))*v*vx
        ay = -g -(rho*CdA/(2*m))*v*vy
        return [vx, vy, ax, ay]

    def hit_ground(t, y):
        return y[1]  # event when y=0
    hit_ground.terminal = True
    hit_ground.direction = -1

    sol = integrate.solve_ivp(ode, (0, 10), [0, 0, vx0, vy0],
                              events=hit_ground, max_step=0.02, rtol=1e-7, atol=1e-9)
    x, y = sol.y[0], sol.y[1]
    t = sol.t
    return t, x, y

def range_for_angle(theta_deg):
    t, x, y = flight(theta_deg)
    return x[-1]

# Optimize angle to maximize range (negative for minimize_scalar)
obj = lambda th: -range_for_angle(th)
res = optimize.minimize_scalar(obj, bounds=(5, 85), method="bounded")
theta_opt = res.x
R_opt = range_for_angle(theta_opt)

# Plot a few trajectories around optimum
angles = [theta_opt-10, theta_opt, theta_opt+10]
fig, ax = plt.subplots()
for th in angles:
    _, x, y = flight(th)
    ax.plot(x, y, label=rf"$\theta={th:.1f}^\circ$")

ax.set_xlabel("x (m)")
ax.set_ylabel("y (m)")
ax.set_title(rf"Trajectories with quadratic drag (optimal θ ≈ {theta_opt:.1f}°, range ≈ {R_opt:.1f} m)")
ax.legend()
plt.show()

# Range vs angle curve
ths = np.linspace(10, 80, 25)
ranges = np.array([range_for_angle(th) for th in ths])

fig, ax = plt.subplots()
ax.plot(ths, ranges, "-o")
ax.axvline(theta_opt, linestyle="--")
ax.set_xlabel(r"$\theta$ (deg)")
ax.set_ylabel("Range (m)")
ax.set_title("Range vs launch angle (with drag)")
plt.show()


# %% [markdown]
### שאלות רב־ברירה (דוגמה 2)
1. לשם עצירת ה־ODE בעת פגיעה בקרקע ($y=0$), השתמשנו ב־  
   A. `events` עם פונקציה שמחזירה $y$  
   B. `curve_fit`  
   C. `root_scalar`  
   D. `stats.norm`  
2. ללא גרר, הזווית המיטבית היא:  
   A. $90^\circ$  
   B. $0^\circ$  
   C. $45^\circ$  
   D. תלוי ב־$v_0$ בלבד

### איור (סקיצה)
- שרטטו כוחות על הקליע: משקל כלפי מטה וגרר ביחס ישר למהירות ובכיוון מנוגד.


# %% [markdown]
## דוגמה 3 — מתנד מרוסן: זיהוי $\omega_n$ ו־$\zeta$
**רקע:** משוואה: $x'' + 2\zeta\omega_n x' + \omega_n^2 x = 0$  
במקרה מרוסן קל ($0<\zeta<1$): $x(t) = A e^{-\zeta\omega_n t}\cos(\omega_d t+\phi)$, עם $ \omega_d = \omega_n\sqrt{1-\zeta^2} $ ו־$ Q=\dfrac{1}{2\zeta} $.

**מטרה:**  
1. ליצור סיגנל מרוסן עם רעש.  
2. לזהות $\omega_d$ מבדיקת FFT/פסגה.  
3. להתאים את המודל המלא עם `curve_fit` כדי לאמוד $A,\zeta,\omega_n,\phi$.  
4. להציג שיוריים ולהעריך $Q$.



In [ ]:
# %%
# Damped oscillator: identify ωn and ζ
rng = np.random.default_rng(7)
zeta_true = 0.05
wn_true = 2*np.pi*5.0  # natural rad/s ~ 5 Hz
A_true, phi_true = 1.0, 0.2

fs = 200.0
T = 6.0
t = np.arange(0, T, 1/fs)

# Generate damped signal with noise
wd_true = wn_true*np.sqrt(1 - zeta_true**2)
x_clean = A_true*np.exp(-zeta_true*wn_true*t)*np.cos(wd_true*t + phi_true)
x_noisy = x_clean + 0.05*rng.normal(size=t.size)

# FFT to estimate wd (peak)
Y = fft.rfft(x_noisy)
f = fft.rfftfreq(t.size, d=1/fs)
amp = np.abs(Y)/t.size
pk, _ = signal.find_peaks(amp, height=np.max(amp)*0.3)
f_peak = f[pk[np.argmax(amp[pk])]] if pk.size else f[np.argmax(amp)]
wd_est = 2*np.pi*f_peak

# Nonlinear fit: x(t)=A*exp(-ζ*wn*t)*cos(wd*t+φ) with wd = wn*sqrt(1-ζ^2)
def damped_model(t, A, zeta, wn, phi):
    wd = wn*np.sqrt(max(1e-12, 1 - zeta**2))
    return A*np.exp(-zeta*wn*t)*np.cos(wd*t + phi)

p0 = [1.0, 0.1, wd_est, 0.0]  # rough initial guesses
bounds = ([0, 0, 0, -2*np.pi], [np.inf, 0.99, np.inf, 2*np.pi])

popt, pcov = optimize.curve_fit(damped_model, t, x_noisy, p0=p0, bounds=bounds, maxfev=20000)
A_hat, zeta_hat, wn_hat, phi_hat = popt
perr = np.sqrt(np.diag(pcov))
Q_hat = 1/(2*zeta_hat) if zeta_hat>0 else np.inf

# Plot results
fig, axs = plt.subplots(2, 1, figsize=(6.5, 6), constrained_layout=True)
axs[0].plot(t, x_noisy, ".", ms=3, label="noisy")
axs[0].plot(t, damped_model(t, *popt), "-", label="fit")
axs[0].set_title(rf"Damped oscillator fit: $\zeta={zeta_hat:.3f}\pm{perr[1]:.3f},\ \omega_n={wn_hat:.2f}\pm{perr[2]:.2f}$")
axs[0].set_xlabel("t (s)")
axs[0].set_ylabel("x(t)")
axs[0].legend()

res = x_noisy - damped_model(t, *popt)
axs[1].axhline(0, color="k", lw=1)
axs[1].plot(t, res, "-", lw=1)
axs[1].set_title("Residuals")
axs[1].set_xlabel("t (s)")
axs[1].set_ylabel("residual")

plt.show()

# Spectrum view with detected peak
fig, ax = plt.subplots()
ax.plot(f, amp, label="spectrum")
ax.axvline(f_peak, linestyle="--", label=rf"peak ~ {f_peak:.2f} Hz")
ax.set_xlim(0, 15)
ax.set_xlabel("f (Hz)")
ax.set_ylabel("Amplitude")
ax.set_title("Amplitude spectrum and dominant peak")
ax.legend()
plt.show()

print(f"Estimated parameters:\n A={A_hat:.3f}±{perr[0]:.3f}, zeta={zeta_hat:.4f}±{perr[1]:.4f}, wn={wn_hat:.3f}±{perr[2]:.3f}, phi={phi_hat:.3f}±{perr[3]:.3f}")
print(f"Estimated Q ~ {Q_hat:.2f}")


# %% [markdown]
### שאלות רב־ברירה (דוגמה 3)
1. הקשר בין $\omega_d$ ו־$\omega_n$ הוא:  
   A. $\omega_d = \omega_n$  
   B. $\omega_d = \omega_n\sqrt{1-\zeta^2}$  
   C. $\omega_d = \zeta\omega_n$  
   D. $\omega_d = \dfrac{\omega_n}{\zeta}$  
2. גורם האיכות $Q$ למרוסן קל:  
   A. $Q=2\zeta$  
   B. $Q=\zeta^2$  
   C. $Q=\dfrac{1}{2\zeta}$  
   D. $Q=\dfrac{1}{\zeta}$

### איור (סקיצה)
- סכמת מעטפת דעיכה $A e^{-\zeta\omega_n t}$ והאוסילציה הפנימית בקצב $\omega_d$.


# %% [markdown]
# נספח קצר — טיפים לשמירת תרשימים
- לשימוש במאמרים/דוחות: עדיף **SVG/PDF** (וקטורי) עבור תרשימי קו/טקסט.  
- לגדלים מדויקים: `fig.set_size_inches(width_inches, height_inches)` או `fig, ax = plt.subplots(figsize=(w,h))`.  
- לשילוב ב-Word/Google Docs: PNG ב־300–600 DPI.  
- הוסיפו יחידות ותיאור ברור לצירים, מקרא, וכותרת עניינית.

> כאן המקום לצרף רשימת תשובות לשאלות, אם תרצו, או להשאירן ללמידה עצמית.
